In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from argparse import Namespace
import matplotlib.pyplot as plt
import av2.rendering.vector as vector_plotting_utils
from shapely.geometry import Polygon

from graph_creator.MapGraph import MapGraph
from graph_creator.ActorGraph import ActorGraph

from av2.map.map_api import ArgoverseStaticMap
from av2.datasets.motion_forecasting import scenario_serialization

from av2.datasets.motion_forecasting.viz.scenario_visualization import (
    visualize_scenario
)
from av2.map.map_api import ArgoverseStaticMap
import os 
os.getcwd()


from graph_creator.create_graph import get_scenario_data, plot_scene_at_timestep

In [2]:
from pathlib import Path
# repo_root = Path("/Users/marius/code/graph_coverage")
repo_root = Path("/home/tmuehlen/repos/graph_coverage")
dataroot = repo_root / "argoverse_data" / "train"
print(repo_root)


/home/tmuehlen/repos/graph_coverage


In [3]:
scenario_folders = [f for f in dataroot.iterdir() if f.is_dir()]

# Print number of scenarios found
print(f"Found {len(scenario_folders)} scenarios")

Found 199908 scenarios


In [7]:
# @ Thomas run here for data creation
actor_graphs = []
import pickle

for i, scenario in enumerate(scenario_folders):
    if i >= 1000:
        break
    log_id = scenario.name
    # check if file exists:
    if (repo_root / "actor_graphs" / "argoverse" / f"{log_id}_map_graph.pkl").exists():
        continue
    timestamp = 1.0
    print(i, log_id)
    scenario, map = get_scenario_data(dataroot, log_id)
    G_map = MapGraph.create_from_argoverse_map(map)
    actor_graph = ActorGraph.from_argoverse_scenario(
                                    scenario, 
                                    G_map, 
                                    delta_timestep_s=1.0,
                                    max_distance_lead_veh_m=50,
                                    max_distance_opposite_forward_m=100,
                                    max_distance_opposite_backward_m=10,
                                    max_distance_neighbor_forward_m=50,
                                    max_distance_neighbor_backward_m=20,
                                    max_node_distance_leading=10,
                                    max_node_distance_neighbor=6,
                                    max_node_distance_opposite=6,
    )
    # actor_graphs.append(actor_graph)
    #actor_graph.visualize_actor_graph(t_idx=timestamp, comp_idx=0,  save_path=(repo_root / "plots" / f"{log_id}_actor_graph.png"))
    #visualize_scenario(scenario, map, save_path=(repo_root / f"{log_id}_video_.mp4") )
    #plot_scene_at_timestep(scenario, map, timestamp, save_path=(repo_root / "plots" / f"{log_id}_scene_at_{timestamp}.png"), lane_label=True, actor_graph=actor_graph)
    #G_map.visualize_graph(save_path=(repo_root / "plots" / f"{log_id}_map_graph.png"))
    with open(repo_root / "actor_graphs" / "argoverse" / f"{log_id}_map_graph.pkl", "wb") as f:
        pickle.dump(G_map, f)
    with open(repo_root / "actor_graphs" / "argoverse" / f"{log_id}_actor_graph.pkl", "wb") as f:
        pickle.dump(actor_graph, f)

344 3c5d4132-3e35-4bce-83a0-cc80d1d74037
345 c99d2572-bfbd-4f2d-9584-c59a726f646f
346 ae455ed7-34f8-4ad4-82ae-e93532ae0048
347 779c7b43-0531-432a-909f-b1463db50a21
348 007cff08-b4e5-4da6-bb98-ebc793a8f878
349 a4e820c2-09ec-4160-9b06-a79fe79aa5a9
350 a6b43ce4-ddcd-495d-8b27-ce24cca3bb75
351 a67e18f7-d0fd-4aa0-a2ac-694b3d949823
352 9a12e7be-2760-491f-9b88-451b12491c4d
353 03c5bbb3-bd48-4859-9072-461bf88a2341
354 899a6c87-cbb4-4ea9-8410-b787e8b262a7
355 b44738c3-b357-4a74-bd60-4f4e23fb31df
356 5a1335d5-02d3-45d3-afc3-ca0ef2244f03
357 e00e959b-88de-4aaa-a464-ed942483832a
358 791fc1a2-a8fc-42bb-9d8b-06136328a69e
359 0e656e21-1f8c-4651-8bf5-d58af8bd79b8
360 24f803c5-e1d2-42ab-aa9a-839d59073003
361 0965af26-ec4c-49c1-83ee-2e282f568a1d
362 17e95a03-3992-4c27-bfdc-df675ebe6d65
363 c6f0b7cf-48a4-4ae3-b526-f818cd338433
364 882244e9-c8d9-4613-ad8d-e2d296b807bd
365 1b7b9ba6-df38-486c-8e7a-c4af4f275976
366 6fcfa761-1376-461c-8bc2-ef9da373026a
367 651a20fc-86c6-46e8-93cd-8e1cfce47c66
368 e51fd93b-c0a

KeyboardInterrupt: 

In [5]:
# and now split the graphs into networkx data:
from tqdm import tqdm
argoverse_scns = os.listdir("/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse")
argoverse_scns = [scn.split("_")[0] for scn in argoverse_scns if scn.endswith("_actor_graph.pkl")]
print(argoverse_scns[:3])

for scn in tqdm(argoverse_scns):
    with open(f"/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse/{scn}_actor_graph.pkl", "rb") as f:
        actor_graph = pickle.load(f)
    timestamps = list(actor_graph.actor_graphs.keys())
    for timestamp in timestamps:
        # actor_graph.visualize_actor_graph(timestamp, 0)
        save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_nx/graph_{scn}_{str(timestamp).replace(".", "_")}.pkl'
        with open(save_path, "wb") as file:
            pickle.dump(actor_graph.actor_graphs[timestamp], file)
        


['8bb96116-f0d0-44c6-8409-fd95f6bac5f4', 'e2877db8-1688-4f56-87fd-552fe8ffb9c7', 'f0258267-fe01-44fb-966a-c947dd1fecd3']


100%|██████████| 344/344 [00:11<00:00, 29.35it/s]


In [ ]:
# actor_graphs = []
# import pickle

# for i, scenario in enumerate(scenario_folders):
#     if i >= 5:
#         break
# #if True: 
#     #log_id = "d25d1aaa-8bb2-4c6b-98a5-fa5aa2ddd2eb"
#     log_id = scenario.name
#     print(log_id)
#     timestamp = 1.0
#     scenario, map = get_scenario_data(dataroot, log_id)
#     G_map = MapGraph.create_from_argoverse_map(map)
#     actor_graph = ActorGraph.from_argoverse_scenario(
#                                     scenario, 
#                                     G_map, 
#                                     delta_timestep_s=1.0,
#                                     max_distance_lead_veh_m=50,
#                                     max_distance_opposite_forward_m=100,
#                                     max_distance_opposite_backward_m=10,
#                                     max_distance_neighbor_forward_m=50,
#                                     max_distance_neighbor_backward_m=20,
#                                     max_node_distance_leading=10,
#                                     max_node_distance_neighbor=6,
#                                     max_node_distance_opposite=6,
#     )
#     actor_graphs.append(actor_graph)
#     actor_graph.visualize_actor_graph(t_idx=timestamp, comp_idx=0,  save_path=(repo_root / "plots" / f"{log_id}_actor_graph.png"))
#     #visualize_scenario(scenario, map, save_path=(repo_root / f"{log_id}_video_.mp4") )
#     plot_scene_at_timestep(scenario, map, timestamp, save_path=(repo_root / "plots" / f"{log_id}_scene_at_{timestamp}.png"), lane_label=True, actor_graph=actor_graph)
#     G_map.visualize_graph(save_path=(repo_root / "plots" / f"{log_id}_map_graph.png"))
#     #with open(repo_root / "data" / f"{log_id}_map_graph.pkl", "wb") as f:
#     #    pickle.dump(G_map, f)
#     #with open(repo_root / "data" / f"{log_id}_actor_graph.pkl", "wb") as f:
#     #    pickle.dump(actor_graph, f)


In [ ]:
import pickle
# Save actor graphs and map data to pickle files
output_dir = repo_root / "data"
output_dir.mkdir(exist_ok=True)

# Save actor graphs
actor_graphs_file = output_dir / "actor_graph_argoverse.pkl"
with open(actor_graphs_file, "wb") as f:
    pickle.dump(actor_graphs, f)

# Save map data for each scenario
map_file = output_dir / f"map_argoverse.pkl"
with open(map_file, "wb") as f:
    pickle.dump(G_map, f)



In [ ]:
from graph_creator.SubgraphExtractor import SubgraphExtractor
import os

# Example usage of the SubgraphExtractor
def analyze_actor_graphs(actor_graphs, strategy='frequency', output_dir=None):
    """
    Analyze actor graphs to find common subgraphs using the specified strategy.
    
    Args:
        actor_graphs: List of ActorGraph objects
        strategy: Strategy for selecting subgraphs
        output_dir: Directory to save visualizations
    
    Returns:
        SubgraphExtractor instance
    """
    
    # Initialize extractor with specified strategy
    extractor = SubgraphExtractor(
        min_subgraph_size=3,
        max_subgraph_size=5, 
        # Custom matchers can be defined here if needed
    )
    
    # Extract subgraphs
    subgraph_library = extractor.extract_subgraphs(actor_graphs[:10])
    
    # Print properties
    extractor.print_subgraph_properties()
    
    # Visualize subgraphs
    if output_dir is not None:
        strategy_dir = os.path.join(output_dir, f"strategy_{strategy}")
        extractor.visualize_subgraphs(output_dir=strategy_dir)
    else:
        extractor.visualize_subgraphs()
    
    return extractor

# Create visualization output directory
output_dir = repo_root /  "plots" / "subgraph_analysis"
os.makedirs(output_dir, exist_ok=True)

# Analyze with different strategies
strategies = ['frequency', 'density', 'coverage', 'complexity', 'size']
extractors = {}

for strategy in strategies:
    extractors[strategy] = analyze_actor_graphs(actor_graphs, strategy, output_dir)

# Example of how to access the subgraphs for a specific actor graph and timestamp
print("\nExample of subgraph decomposition:")
print("=================================")
example_graph_idx = 0  # First graph in the list
example_timestamp = list(actor_graphs[example_graph_idx].actor_graphs.keys())[0]
print(f"Graph {example_graph_idx}, Timestamp {example_timestamp}:")

for sg_info in actor_graphs[example_graph_idx].actor_subgraphs[example_timestamp]:
    sg_id = sg_info['subgraph_id']
    nodes = sg_info['nodes']
    print(f"  Subgraph ID: {sg_id}, Nodes: {nodes}")
    # Additional information can be accessed from sg_info['node_mapping']